In [5]:
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

internal_urls = set()
external_urls = set()

def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

def get_all_website_links(url):
    urls = set()
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
        
    for a_tag in soup.findAll("a"):
        
        href = a_tag.attrs.get("href")
        
        if href == "" or href is None:
            # href empty tag
            continue
        # join the URL if it's relative (not absolute link)
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        # remove URL GET parameters, URL fragments, etc.
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_valid(href):
            # not a valid URL
            continue
        if href in internal_urls:
            # already in the set
            continue
        if domain_name not in href:
            # external link
            if href not in external_urls:
                print(f"[!] External link: {href}")
                external_urls.add(href)
            continue
        print(f"[*] Internal link: {href}")
        urls.add(href)
        internal_urls.add(href)
        
    print("[+] Total Internal links:", len(internal_urls))
    print("[+] Total External links:", len(external_urls))

get_all_website_links('https://www.online-bouwmaterialen.nl/')

[*] Internal link: https://www.online-bouwmaterialen.nl/checkout/cart/
[*] Internal link: https://www.online-bouwmaterialen.nl/
[*] Internal link: https://www.online-bouwmaterialen.nl/bezorgkosten
[!] External link: https://www.kiyoh.com/reviews/1045277/online-bouwmaterialen_nl
[*] Internal link: https://www.online-bouwmaterialen.nl/contacts
[*] Internal link: https://www.online-bouwmaterialen.nl/hout-plaatmateriaal/
[*] Internal link: https://www.online-bouwmaterialen.nl/stenen/
[*] Internal link: https://www.online-bouwmaterialen.nl/grind-zand/
[*] Internal link: https://www.online-bouwmaterialen.nl/vbi-broodjesvloer-vbi-renovatievloer-vbi-combinatievloer/
[*] Internal link: https://www.online-bouwmaterialen.nl/isolatiemateriaal/
[*] Internal link: https://www.online-bouwmaterialen.nl/cement-beton/
[*] Internal link: https://www.online-bouwmaterialen.nl/bouwstaalmatten-betonstaal-staaf-zwaluwstaartplaten/
[*] Internal link: https://www.online-bouwmaterialen.nl/gipskartonplaten-gipspl